# 9、代理的使用

---
## 9、1 代理的设置

---
前面介绍了多种请求库，如requests、urllib、Selenium等。接下来我们先贴近实战，了解一下代理如何使用，为后面了解代理池、ADSL拨号代理的使用打下基础。

---
首先，我们来梳理一下这些请求库的代理设置方法。

---

### 1、获取代理

---
做测试之前，我们需要先获取一个可用代理。搜索引擎搜索“代理”关键字，就可以看到许多代理服务网站，网站上会有很多免费代理，比如西刺：http://www.xicidaili.com/ 。 但是这些免费代理大多数情况下都是不好用的，所以比较靠谱的方法是购买付费代理。付费代理在很多网站上都有售卖，数量不用多，稳定可用即可，我们可以自行选购。

---
如果本机有相关代理软件的话，软件一般会在本机创建HTTP或SOCKS代理服务，本机直接使用此代理也可以。

---
本书中作者，在其本机安装了一部代理软件，它会在本地9743端口上创建HTTP代理服务，即代理为127.0.0.1:9743，另外还会在9742端口创建SOCKS代理服务，即代理为127.0.0.1:9742.他只要设置了这个代理，就可以成功将本机IP切换到代理软件连接的服务器的IP了。

---
本章下面的示例里，他使用上述代理来演示其设置方法，我们也可以自行替换成自己的可用代理。设置代理后测试的网址是： http://httpbin.org/get ， 我们访问该网址就可以得到请求的相关信息，其中origin字段就是客户端的IP，我们可以根据它来判断代理是否设置成功，即是否成功伪装了IP。

### 2、urllib

---
首先，我们以最基础的urllib为例，来看一下代理的设置方法，代码示例如下：

In [2]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener

proxy = '127.0.0.1:9743'
proxy_handler = ProxyHandler({
    'http': 'http://' + proxy,
    'https': 'https://' + proxy
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('http://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

[WinError 10061] 由于目标计算机积极拒绝，无法连接。


这里我们需要借助ProxyHandler设置代理，参数是字典类型，键名是字典类型，键名为协议类型，键值是代理。注意，此处代理前面需要加上协议，即http或者https。当请求的链接是http协议时，ProxyHandler会调用http代理。当请求的链接是https的时候，会调用https代理。此处生效的代理是http://127.0.0.1:9743 .

---
创建完ProxyHandler对象之后，我们需要利用build_opener()方法传入该对象来创建一个Opener，这样就相当于此Opener已经设置好代理了。接下来直接调用Opener对象的open()方法，即可访问我们所想要的链接。

---
运行输出结果是一个JSON，它有一个字段origin，标明了客户端的IP。验证一下，此处的IP确实为代理的IP，并不是真实的IP。这样我们就成功设置好代理，并可以隐藏真实IP了。

---
如果遇到需要认证的代理，我们可以通过如下方法设置：

In [ ]:
from urllib.error import URLError
from urllib.request import ProxyHandler, build_opener

proxy = 'username:password@127.0.0.1:9743'
proxy_handler = ProxyHandler({
    'http': 'http://' + proxy,
    'https': 'https://' + proxy
})
opener = build_opener(proxy_handler)
try:
    response = opener.open('http://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

这里改变的只是proxy变量，我们只需要在代理前面加入代理认证的用户名和密码即可。其中，username为用户名，password为密码，例如，username为foo，密码为bar，那么代理就是foo:bar@127.0.0.1:9743。

---
如果代理是SOCKS5类型，那么我们可以用如下方式设置代理：

In [4]:
import socks
import socket
from urllib import request
from urllib.error import URLError

socks.set_default_proxy(socks.SOCKS5, '127.0.0.1', 9742)
socket.socket = socks.socksocket
try:
    response = request.urlopen('http://httpbin.org/get')
    print(response.read().decode('utf-8'))
except URLError as e:
    print(e.reason)

Socket error: Connection closed unexpectedly


本地有一个SOCKS5代理，它运行在9742端口。运行成功之后，输出结果和上文HTTP代理的输出结果一样。

---
此结果中的origin字段同样为代理的IP。

### 3、requests

---
对于requests来说，代理设置更加简单，我们只需要传入proxies参数即可。

In [ ]:
import requests

# proxy = '127.0.0.1:9743'
proxy = 'cj424k:cj424k@120.210.204.164:888'
proxies = {
    'http': 'http://' + proxy,
    'https': 'https://' + proxy,
}
try:
    response = requests.get('http://httpbin.org/get', proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error', e.asrgs)



requests只需要构造代理字典，然后通过proxies参数即可，而不需要重新构建Opener。

---
如果需要使用SOCKS5代理

In [ ]:
import requests

# proxy = '127.0.0.1:9743'
proxy = 'cj424k:cj424k@120.210.204.164:888'
proxies = {
    'http': 'socks5://' + proxy,
    'https': 'socks5://' + proxy
}
try:
    response = requests.get('http://httpbin.org/get', proxies=proxies)
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error', e.asrgs)


---

全局设置的SOCKS5方法

In [ ]:
import requests
import socks
import socket

socks.set_default_proxy(socks.SOCKS, '127.0.0.1', 9742)
socket.socket = socks.socksocket
try :
    response = requests.get('http://httpbin.org/get')
    print(response.text)
except requests.exceptions.ConnectionError as e:
    print('Error', e.asrgs)

### 4、Selenium

---
Selenium同样也可以设置代理，包括两种方式：一种是有界面浏览器，以Chrome为例；另一种是无界面浏览器，以PhantomJS为例。

---

+ 对于Chrome来说，用Selenium设置代理的方法也非常简单，设置方法如下：

In [7]:
from selenium import webdriver

proxy = '127.0.0.1:9743'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=http://' + proxy)
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get('http://httpbin.org/get')


C:\anoconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


WebDriverException: Message: Can not connect to the Service chromedriver


如果代理是认证代理，则比较麻烦：

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import zipfile


ip = '120.210.204.164'
port = 888
username = 'cj424k'
password = 'cj424k'

mainfest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["backgroud.js"]
    }
}
"""

backgroud_js = """
    var config = {
        mode: "fixed_servers",
        rules: {
            singleProxy:{
                scheme: "http",
                host: "%(ip)s",
                port: %(port)s
            }
        }
    }
    chrome.proxy.settings.set({value: config, scope: "regular"}, function(){});
    function callbackFn(details){
        return {
            authCredentials: {
                username: "%(username)s",
                password: "%(password)s"
            }
        }
    }
    chrome.webRequest.onAuthRequired.addListener(
        callbackFn,
        {urls: ["<all_urls>"]},
        ['blocking']
    )
""" % {'ip': ip, 'port': port, 'username': username, 'password': password}

plugin_file = 'proxy_auth_plugin.zip'
with zipfile.ZipFile(plugin_file, 'w') as zp:
    zp.writestr("manifest.json", mainfest_json)
    zp.writestr("backgroud.js", backgroud_js)
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_extension(plugin_file)
browser = webdriver.Chrome(chrome_options=chrome_options)
browser.get('http://httpbin.org/get')

这里需要在本地创建一个manifes.json配置文件和background.js脚本来设置认证代理。运行代码之后本地会生成一个proxy_auth_plugin.zip文件来保存当前配置。

---

+ PhantomJS

---
对于PhantomJS来说，代理设置方法可以借助service_args参数，也就是命令行参数。代理设置方法如下：

In [ ]:
from selenium import webdriver

service_args = [
    '--proxy=127.0.0.1:9743',
    '--proxy-type=http'
]
browser = webdriver.PhantomJS(service_args=service_args)
browser.get('http://httpbin.org/get')
print(browser.page_source)


如果需要认证，那么只需要再加入--proxy-auth选项即可，这样参数就改为：

In [ ]:
service_args = [
    '--proxy=127.0.0.1:9743',
    '--proxy-type=http',
    '--proxy-auth=username:password'
]

# 将username和password替换为认证所需的用户名和密码即可。